<a href="https://colab.research.google.com/github/rahiakela/machine-learning-research-and-practice/blob/main/ensemble-methods-for-machine-learning/01-ensemble-methods/model_averaging_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##A Simple Model Averaging Ensemble

We will build our first ensemble with model averaging for a regression task on the classic benchmark **Boston Housing** data set. 

First, we load the data set and then standardize the data to be zero-mean, unit standard deviation, and the labels to be in the range [-1, 1]. 

After standardization, split into training and test sets (67%-33%).

##Setup

In [ ]:
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score

from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.neural_network import MLPRegressor

from itertools import combinations
import numpy as np


import matplotlib.pyplot as plt
%matplotlib inline

##Dataset